# 1. Simplitied Desicion Tree

In [31]:
import numpy as np
import pandas as pd

class DecisionTree:
    def __init__(self):
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _entropy(self, y): 
        counts = np.bincount(y)  # Count the occurrences of each class
        probs = counts / len(y)  # Calculate class probabilities
        return -np.sum(probs * np.log2(probs + 1e-6))  # Compute entropy

    def _information_gain(self, y, splits): 
        total = len(y)
        entropy_y = self._entropy(y)  # Entropy of y
        entropy_splits = 0

        for split in splits:
            weight = len(split) / total  # Weight of the split
            entropy_splits += weight * self._entropy(split)  # Entropy of each splits

        return entropy_y - entropy_splits
    
    def _split_information(self, splits):
        total = len(np.concatenate(splits))
        split_info = 0

        for split in splits:
            if len(split) > 0:
                p_i = len(split) / total
                split_info -= p_i * np.log2(p_i)

        return split_info

    def _best_split(self, X, y):
        best_gain_ratio = 0
        best_feature = None
        best_threshold = None

        for feature in range(X.shape[1]):
            unique_values = np.unique(X[:, feature])  # thredshold is on training points
            for threshold in unique_values: 
                left_mask = X[:, feature] >= threshold
                right_mask = ~left_mask

                if len(y[left_mask]) > 0 and len(y[right_mask]) > 0:
                    splits = [y[left_mask], y[right_mask]]
                    gain = self._information_gain(y, splits)  # Information gain for current thredshold
                    split_info = self._split_information(splits)
                    gain_ratio = gain / (split_info + 1e-6)

                    if gain_ratio > best_gain_ratio:
                        best_gain_ratio = gain_ratio
                        best_feature = feature
                        best_threshold = threshold

        return best_feature, best_threshold

    def _build_tree(self, X, y):
        
        # the entropy of any candidates split is zero
        if len(np.unique(y)) == 1:
            return int(y[0])  # Return a leaf node with the class if all labels are the same

        # the node is empty
        if len(X) == 0:
            return int(1)  # Predict y = 1 when no majority class

        feature, threshold = self._best_split(X, y)

        # all splits have zero gain ratio
        if feature is None:
            return int(1)  # Predict y = 1 when no majority class

        left_mask = X[:, feature] >= threshold
        right_mask = ~left_mask

        left_subtree = self._build_tree(X[left_mask], y[left_mask])  # Recursively build left subtree
        right_subtree = self._build_tree(X[right_mask], y[right_mask])  # Recursively build right subtree

        return (int(feature), threshold, left_subtree, right_subtree)  # Return a decision node

    def predict(self, X):
        predictions = []
        for x in X:
            node = self.tree
            while isinstance(node, tuple):
                feature, threshold, left, right = node
                if x[feature] >= threshold:
                    node = left  # Traverse left subtree
                else:
                    node = right  # Traverse right subtree
            predictions.append(node)

        return np.array(predictions)
    
    def visualize_tree(self):
        self._visualize_node(self.tree, depth=0)

    def _visualize_node(self, node, depth):
        if isinstance(node, int):
            print(f"{'  ' * depth}Class {node}")
        else:
            feature, threshold, left, right = node
            print(f"{'  ' * depth}Feature {feature} >= {threshold}")
            self._visualize_node(left, depth + 1)
            self._visualize_node(right, depth + 1)

# 2. Questions

## 2-2. Our algorithm is greedy

In [33]:
X = np.array([[1.2, 2.3], [2.1, 1.9], [3.5, 4.0]])
y = np.array([0, 1, 0])

tree = DecisionTree()
tree.fit(X, y)

test_data = np.array([[2.8, 3.2], [1.0, 2.0]])
predictions = tree.predict(test_data)

print(predictions)

[0 1]


## 2-3. Information gain ration exercise

In [9]:
druns = pd.read_csv('./Homework 2 data/Druns.txt', sep=" ", header=None)

## 2-4. The king of interpretabilit

In [32]:
d3leaves = pd.read_csv('./Homework 2 data/D3leaves.txt', sep=" ", header=None)

tree_d3leaves = DecisionTree()
tree_d3leaves.fit(np.array(d3leaves[[0, 1]]), np.array(d3leaves[2]))

tree_d3leaves.visualize_tree()

Feature 0 >= 10
  Class 1
  Feature 1 >= 3
    Class 1
    Class 0
